# DYSKUSJA

PYTANIE: Czego potrzebujemy?
ODPOWIEDZ: Uczenie nienadzorowanego!

PYTANIE: Czemu?
ODPOWIEDZ: ...

PYTANIE: Czego użyjemy?
ODPOWIEDZ: Używamy GAN.

PYTANIE: Czemu GAN?
ODPOWIEDZ: Ponieważ ogólna zasada jest taka, że próbujemy coś wygenerować i mamy detykowanego agenta, który mówi czy to się udało czy nie.
Brzmi ok do uczenia generowania obrazów już istniejących.

PYTANIE: W takim razie jak to działa że tworzy nowe obrazy?
ODPOWIEDZ: Mamy w sumie dwa modele, jeden ma generować np. kwiatki, a drugi ma mówić czy to co dostaje do kwiatek. Jak model rozpoznający kwiatki powie "to nie kwiatek" to model generujący się updatuje, a jak model rozpoznający powie "to totalnie jest kwiatek" na to co dał model generujący, to model rozpoznający się updatuje i tak w kółko...
Na sam koniec mamy model, który dostając obraz na wejście "przewiduje" co może być na wyjściu, czyli na nasze, będzie tworzył obraz jaki "być powinien" na bazie tego czego się nauczył.

PYTANIE: A jakie sieci w tym GAN pan ma?
ODPOWIEDZ: convolutional neural network (CNN)

PYTANIE: Czemu?
ODPOWIEDŹ: CNN mogą uczyć się na "historii" czyli trochę tak jakby to był film o kwiatkach i model będzie przewidywał następną klatkę tego filmu o kwiatkach. Żeby to zrobić musi coś wygenerować i nam zaproponować. To tutaj mamy część "twórczą".


# SCHEMAT ROZWIĄZANIA

Narazie ta część bazuje/jest z książki "Deep Learning Praca z językiem Python i biblioteką Kares" autorstwa Francois Chollet:
1. Sieć generatora "generator" mapuje wektor o kształcie "latent_dim" na obraz o kształcie (32, 32, 3).
2. Sieć dyskryminatora "discriminator" mapuje obraz o kształcie (32, 32, 3) na binarną wartość określającą prawdopodobieństwo tego, że obraz jest prawdziwy.
3. Sieć "gan" tworzy łańcuch  skłądajaćy się z generatora i dyskryminatora (gan(x)=discriminator(generator(x))). SIeć gan mapuje wektory niejawnej przestrzeni na oceny realizmu wystawiane przez dyskryminator.
4. Trenujemy dyskryminator przy użyciu przykładów prawdziwych i wygenerowanych przez generator, oznaczonych etykietami, tak jakbyśmy trenowali zwykły model klasyfikacji obrazów.
5. W celu wytrenowania generatora korzystamy z gradientów wag generatora w odniesieniu do straty modelu "gan". 
Inaczej (rozwijajac): W zwiazku z tym każdy krok trenowania ma modyfikować wagi generatora tak, aby zwiększyć prawdopodobieństwo zaklasyfikowania wygenerowanych obrazów jako prawdziwych. 
Inaczej (upraszczająć): Trenujemy generator tak by był w stanie oszukać dyskryminator.

# IMPORT BIBLIOTEK

In [23]:
import keras
from keras import layers
import numpy as np

# WARTOŚĆI

In [24]:
# Zmienne opisujące obrazy
latent_dim = 32
height = 32
width = 32
channels = 3

# GENERATOR

Do stworzenia modelu generatora użyjemy konwoluncyjnej sieci neuronowej (ang. Convolutional Neural Network - CNN).

Wybrałem ten rodzaj sieci ze względu na jej dużą efektywność rozwiązywaniu problemów widzenia maszynowego.
Jej efektywność polega na tym, że w przeciwieństwie do klasycznych sieci gęstych (ang. Dense), które uczą się wzorców globalnych, sieci konwoluncyjne uczą się wzorców lokalnych (np. krawędzie, zaokrąglenia itd.). Kolejne warstwy rozpoznają coraz bardziej skomplikowane wzorce (np. uszy, nosy itp.).

Skoro konwoluncyjna sieć neuronowa radzi sobie z rozpoznawaniem obrazów, może będzie dobrze działać przy ich generowaniu? 

In [25]:
# INPUT LAYER: Warstwa wejściowa generatora
generator_input = keras.Input(shape=(latent_dim,))

# HIDDEN LAYERS: Pozostałe warstwy modelu generatora, uczące się wzorców obrazów (CNN)
# CO ROBIMY? 
# BO: Zmieniamy obiekt wejściowy w 128 kanałową mapę cech 16x16
# CZEMU TO TU JEST?
# BO: By gęstwa warstwa sieci neuronowej mogła przetworzyć obraz
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)  # Tworzymy 

# CO ROBIMY?
# BO: Standardowo trenujemy rozpoznawanie obrazów
# CZEMU TO TU JEST?
# BO: Conv2D to 2 wymiarowa sieć konwoluncyjna, generalnie dająca dobre wyniki w rozpoznawaniu obrazów
# CZEMU TAKIE WARTOŚĆI?
# BO: 
# 256 kanałów, byśmy mogli wyłapać jak najwięcej cech.
# 5 oznaczająca 5x5 mapa cech, ponieważ przy mniejszej moglibyśmy nie wyłapać bardziej ogólnych cech na których nam zależy
# padding ustawiony na 'same' oznacza tyle, że włączamy padding. Padding to dopisywanie 0 z prawej/lewej lub dołu/góry do wektora.
# "same" results in padding with zeros evenly to the left/right or up/down of the input. 
x = layers.Conv2D(256, 5, padding='same')(x)
X = layers.LeakyReLU()(x)

# CZEMU TO TU JEST?
# BO: Chcemy zwiększyć odrobinę obraz by lepiej wyłapać jego cechy.
# When padding="same" and strides=1, the output has the same size as the input.
x = layers.Conv2D(256, 4, strides=2, padding='same')(x)  # Zwiększenie rozmiaru do 32x32. Jeśli padding jest włączony (ustawiony na 'same') i strides jest ustawione na 1 to obraz wyjściowy jest takiego samego rozmiaru co wejsciowy. Przy sutawieniu na 2 obraz wyjściowy jest dwukrotnie większy jak wejściowy.
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# OUTPUT LAYER: Warstwa wyjściwowa generatora, która daje nam obraz. 
# Tworzy instancję generatora, która mapuje obiekt wejściowy o kształcie (latent_dim,) na obraz o kształcie (32, 32, 3)
# CZEMU TANH?
# BO: tanh to "Hyperbolic tangent activation function". DUNNO ;_______; 
# JAK TO DZIAŁA?
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

# DEKLARACJA MODELU GENERATORA
# JAK TO GENERUJE OBRAZ?
# BO: ...
generator = keras.models.Model(generator_input, x)  # Generuje jednokanałową mapę cech o rozmiarze 32x32 (rozmiar ten jesy taki sam jak rozmiar obraz,ow wchodzących w skład zbioru CIFAR10)
generator.summary()

# OPTYMALIZACJA MODELU GENERATORA
# nie ma takiej potrzeby, ponieważ model generatora jest zawarty w modelu dyskryminatora
# wystarczy że ustawimy optymalizatora dla modelu dyskryminatora.
# KOMPILACJA MODELU GENERATORA
# j/w

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_7 (Dense)             (None, 32768)             1081344   
                                                                 
 leaky_re_lu_33 (LeakyReLU)  (None, 32768)             0         
                                                                 
 reshape_5 (Reshape)         (None, 16, 16, 128)       0         
                                                                 
 conv2d_32 (Conv2D)          (None, 16, 16, 256)       819456    
                                                                 
 conv2d_33 (Conv2D)          (None, 8, 8, 256)         1048832   
                                                                 
 leaky_re_lu_35 (LeakyReLU)  (None, 8, 8, 256)         0   

# Dyskryminator

Uzasadnienie podobne jak do generatora.

In [26]:
# INPUT LAYER
discriminator_input = layers.Input(shape=(height, width, channels))

# HIDDEN LAYERS
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)  # Powiększamy obraz by nie stracić szczegółów przy ocenie prawidziwości obrazka
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)  # j/w
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)  # j/w
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)  # Ważna warstwa bo GAN łątwo wpada w optimum lokalne przy doborze wag

# OUTPUT LAYER
x = layers.Dense(1, activation='sigmoid')(x)  # klasyczna warstwa klasyfikacji binarnej

# DEKLARACJA MODELU DYSKRYMINATORA
# Tworzenie instancji modelu dyskryminatora zamieniajacego obiekt wejściowy mający kształt (32, 32, 3) na wynik klasyfikacji binarnej określającej prawdziwość obrazu
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()


Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_37 (Conv2D)          (None, 30, 30, 128)       3584      
                                                                 
 leaky_re_lu_38 (LeakyReLU)  (None, 30, 30, 128)       0         
                                                                 
 conv2d_38 (Conv2D)          (None, 14, 14, 128)       262272    
                                                                 
 leaky_re_lu_39 (LeakyReLU)  (None, 14, 14, 128)       0         
                                                                 
 conv2d_39 (Conv2D)          (None, 6, 6, 128)         262272    
                                                                 
 leaky_re_lu_40 (LeakyReLU)  (None, 6, 6, 128)         0   

In [27]:
# OPTYMALIZACJA MODELU DYSKRYMINATORA
discriminator_optimizer = keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,  # Optymalizator korzytsa z mechanizmu ucinania wartości gradientu
    decay=1e-8  # W celu uzyskania stabilnego przebiegu procesu trenowania korzystamy z parametru rozkłądu współczynnika uczenia
)

c:\Users\jedrz\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [28]:
# KOMPILACJA MODELU DYSKRYMINATORA
discriminator.compile(
    optimizer=discriminator_optimizer,
    loss='binary_crossentropy'
)

# SIEĆ Z PRZECIWNIKIEM

In [31]:
discriminator.trainable = False  # Umożliwiamy trenowanie wag dyskryminatora (tylko w modelu gan)

# INPUT LAYER
gan_input = keras.Input(shape=(latent_dim,))

# HIDDEN LAYERS
pass

# OUTPUT LAYER
gan_output = discriminator(generator(gan_input))

# DEKLARACJA MODELU GAN
gan = keras.models.Model(gan_input, gan_output)


ValueError: Exception encountered when calling layer "conv2d_39" (type Conv2D).

Negative dimension size caused by subtracting 4 from 2 for '{{node model_7/conv2d_39/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 2, 2, 1], use_cudnn_on_gpu=true](model_7/leaky_re_lu_39/LeakyRelu, model_7/conv2d_39/Conv2D/ReadVariableOp)' with input shapes: [?,2,2,128], [4,4,128,128].

Call arguments received by layer "conv2d_39" (type Conv2D):
  • inputs=tf.Tensor(shape=(None, 2, 2, 128), dtype=float32)

In [ ]:
# OPTYMALIZACJA MODELU GAN
gan_optimizer = keras.optimizers.RMSprop(
    lr=0.0004,
    clipvalue=1.0,
    decay=1e-8
)

In [ ]:
# KOMPILACJA MODELU GAN
gan.compile(
    optimizer=gan_optimizer,
    loss='binary_crossentropy'
)

# NOTATKI
## Rule of thumbs
1. Ostatnia warstwa to *tanh*.
2. Próbkowanie punktów za pomocą rozkłądu Gaussa.
3. Dodajemy dużo losowości do procesu losowania (bo GAN ma tendencję do wpadania w optimum lokalne, BARDZO), np. drop wag i szum etykiet.
4. Rzadkie gradienty są fe. Co robi rzadki gradient? Np. Maxpooling i ReLU. Wiec zamiast maxpooling dajemy krokową konwolucję. Zamiast ReLU dajemy LeakyReLU.
5. Zawsze gdy używamy Conv2DTranpose lub Conv2D zastosujemy rozmiar jądra podzielony przez rozmiar kroku. POmoże nam to uniknąć artefaktów takich jak "szachownica" na generowanym obrazie.